In [300]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
# Importing Classifier Modules
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
%matplotlib inline
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
le = LabelEncoder()
train['Name'] = le.fit_transform(train['Name'])
train['Sex'] = le.fit_transform(train['Sex'])
train['Ticket'] = le.fit_transform(train['Ticket'])
train['Cabin'] = le.fit_transform(train['Cabin'])
train['Embarked'] = le.fit_transform(train['Embarked'])

test['Name'] = le.fit_transform(test['Name'])
test['Sex'] = le.fit_transform(test['Sex'])
test['Ticket'] = le.fit_transform(test['Ticket'])
test['Cabin'] = le.fit_transform(test['Cabin'])
test['Embarked'] = le.fit_transform(test['Embarked'])
# Удаляем строки с пустыми значениями
train_excluded_rows = train.dropna()
test_excluded_rows = test.dropna()
# Удаляем строки с пустыми стольбцами
train_excluded_cols = train.dropna(axis=1)
test_excluded_cols = test.dropna(axis=1)

In [301]:
# Заменяем null на моды
modes = train.mode().iloc[0]
train_replaced_medians = train.fillna(modes)

modes = test.mode().iloc[0]
test_replaced_medians = test.fillna(modes)

In [302]:
# Предсказываем пустые значения
train_encoded = pd.get_dummies(train)
test_encoded = pd.get_dummies(test)
le = LabelEncoder()
imputer = KNNImputer(n_neighbors=20)
train_imputed = pd.DataFrame(imputer.fit_transform(train_encoded), columns=train_encoded.columns)
test_imputed = pd.DataFrame(imputer.fit_transform(test_encoded), columns=test_encoded.columns)
clf = KNeighborsClassifier(n_neighbors=30)

In [303]:
X_train = train_excluded_rows.drop('Survived', axis=1)
y_train = train_excluded_rows['Survived']

X_test = test.dropna().copy()


In [304]:
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_train, y_train) * 100, 2)
print (str(acc_log_reg) + ' percent')

65.97 percent


In [305]:
X_train = train_excluded_cols.drop('Survived', axis=1)
y_train = train_excluded_cols['Survived']

X_test = test.copy().drop(['Age'],axis=1).dropna()

In [306]:
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_train, y_train) * 100, 2)
print (str(acc_log_reg) + ' percent')

67.34 percent


In [307]:
X_train = train_replaced_medians.drop('Survived', axis=1)
y_train = train_replaced_medians['Survived']

X_test = test_replaced_medians.copy()

In [308]:
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_train, y_train) * 100, 2)
print (str(acc_log_reg) + ' percent')

67.12 percent


In [309]:
X_train = train_imputed.drop('Survived', axis=1)
y_train = train_imputed['Survived']

X_test = test_imputed.copy()

In [310]:
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_train, y_train) * 100, 2)
print (str(acc_log_reg) + ' percent')

67.34 percent


In [311]:
percentages = pd.DataFrame({
    'Operation': ['Deletion of Rows', 'Deletion of Columns', 'Mode Imputation', 'Prediction of Values'],
    'Percentage': ['65.97 percent', '67.34 percent', '67.12 percent', '67.34 percent']
})
print(percentages)


              Operation     Percentage
0      Deletion of Rows  65.97 percent
1   Deletion of Columns  67.34 percent
2       Mode Imputation  67.12 percent
3  Prediction of Values  67.34 percent


In [312]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": y_pred_log_reg
    })

In [313]:
submission.to_csv('submission.csv', index=False)